In [1]:
using Random
using Distributions
using DataFrames, CSV

# compile local files
include("statistics.jl")
include("simulation.jl")
include("data.jl")
include("util.jl")

using .TestStatistics
using .Simulation
using .Data

In [2]:
"""
CONFIG
"""

pooled = true
alpha = 0.05
isTwoSided = false

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8      # size of group 2
px, py = partition(nx, ny)
dtype = Float32

# distributions
Random.seed!(123)

distrTypeX = Gamma
shapeX = dtype.(rand(Uniform(1, 5), B))
scaleX = 4 * shapeX

distrTypeY = Gamma
shapeY = dtype.(rand(Uniform(1, 5), B))
scaleY = 4 * shapeY

10-element Vector{Float32}:
 15.945381
 17.107214
 19.266548
 17.53432
 13.3879795
  5.9490023
  6.20426
  4.8129325
 13.273467
 16.86187

In [3]:
# generate data
x, y, wide, narrow, deltas, distrX, distrY = generateData(B, S, nx, ny, pooled,
                                                          distrTypeX, (shapeX, scaleX),
                                                          distrTypeY, (shapeY, scaleY))
@show size(x)
@show size(y)
@show size(wide)
@show size(narrow)
@show deltas[1]
@show distrX[1]
@show distrY[1]

size(x) = (10, 18, 8)
size(y) = (10, 18, 8)
size(wide) = (10, 18)
size(narrow) = (10, 18)
deltas[1] = 22.006042f0
distrX[1] = Gamma{Float32}(α=4.6251984f0, θ=18.500793f0)
distrY[1] = Gamma{Float32}(α=3.9863453f0, θ=15.945381f0)


Gamma{Float32}(α=3.9863453f0, θ=15.945381f0)

In [4]:
# wrap constant arguments into an iterable object
# this allows each batch to have local access

struct Args
    px::AbstractMatrix{Int}
    py::AbstractMatrix{Int}
    pooled::Bool
    alpha::Float32
    alt_lo::Alternative
    alt_hi::Alternative
end

Base.Broadcast.broadcastable(obj::Args) = Ref(obj)  # make objects of this type iterable

In [8]:
for isTwoSided in [true, false]
    alpha_temp = alpha
    alt_lo = alt_hi = Simulation.twoSided
    
    if !isTwoSided
        alpha_temp = alpha / 2
        alt_lo = greater
        alt_hi = smaller
    end
        
    for pooled in [true, false]
        args = Args(px, py, pooled, alpha_temp, alt_lo, alt_hi)
        
        results = coverage.(eachslice(x, dims=1),
                            eachslice(y, dims=1),
                            eachrow(wide),
                            eachrow(narrow),
                            deltas,
                            args)
        
        # convert results to DataFrame
        probs  = [x for (x, _) in results]
        widths = [x for (_, x) in results]
        df = DataFrame(prob=probs, width=widths, distrX=distrX, distrY=distrY)

        # save DataFrame as .csv
        filename = (isTwoSided ? "twoSided" : "oneSided") * "_" *
                   (pooled ? "" : "un") * "pooled" * "_" * string(args.alpha) * ".csv"
        
        #CSV.write("../results/" * filename, df)
        CSV.write(filename, df)
    end
end